In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import log
from collections import Counter

In [120]:
attrib = {'outlook':0,'temperature':1,'humidity':2,'windy':3,'play soccer?':4}
X = np.array([
['sunny',85,85,'false','no'],
['sunny',80,90,'true','no'],
['overcast',83,78,'false','yes'],
['ranny',70,96,'false','yes'],
['ranny',68,80,'false','yes'],
['ranny',65,70,'true','no'],
['overcast',64,65,'true','yes'],
['sunny',72,95,'false','no'],
['sunny',69,70,'false','yes'],
['ranny',75,80,'false','yes'],
['sunny',75,70,'true','yes'],
['overcast',72,90,'true','yes'],
['overcast',81,75,'false','yes'],
['ranny',71,80,'true','no']])

In [121]:
def media_attrib(atributo,X):
    indice = attrib[atributo]
    return np.sum(X[:,indice].astype(float))/len(X)

In [122]:
def discretize(atributo, X):
    indice = attrib[atributo]
    media = media_attrib(atributo,X)
    for linha in X:
        if float(linha[indice]) <= media:
            linha[indice] = "menor"
        else:
            linha[indice] = "maior"
    return X
discretize('temperature', X)  
discretize('humidity', X)

array([['sunny', 'maior', 'maior', 'false', 'no'],
       ['sunny', 'maior', 'maior', 'true', 'no'],
       ['overcast', 'maior', 'menor', 'false', 'yes'],
       ['ranny', 'menor', 'maior', 'false', 'yes'],
       ['ranny', 'menor', 'menor', 'false', 'yes'],
       ['ranny', 'menor', 'menor', 'true', 'no'],
       ['overcast', 'menor', 'menor', 'true', 'yes'],
       ['sunny', 'menor', 'maior', 'false', 'no'],
       ['sunny', 'menor', 'menor', 'false', 'yes'],
       ['ranny', 'maior', 'menor', 'false', 'yes'],
       ['sunny', 'maior', 'menor', 'true', 'yes'],
       ['overcast', 'menor', 'maior', 'true', 'yes'],
       ['overcast', 'maior', 'menor', 'false', 'yes'],
       ['ranny', 'menor', 'menor', 'true', 'no']],
      dtype='<U8')

In [123]:
log2 = lambda x:log(x)/log(2)
HX = 0.0
arvore = {}

In [124]:
def get_entropy(X):
    freq = Counter(X[:,-1])
    tam = len(X)
    entropy = 0
    for f in freq.values():
        pi = f/tam
        if pi > 0:
            entropy -= pi * log2(pi)
    return entropy

In [125]:
def get_entropy_attrib(atributo, X):
    indice = attrib[atributo]
    valores = Counter(X[:,indice])
    #print('valores:{}'.format(valores))
    soma, tam = 0, len(X)
    #print('soma:{}\ttam:{}'.format(soma,tam))
    for valor, freq in valores.items():
        pk = freq / tam
        sub_conjunto = [linha for linha in X if linha[indice] == valor]
        #print(sub_conjunto)
        soma += pk * get_entropy(np.array(sub_conjunto))
    return soma

In [127]:
# Obter a entropia do dataset inteiro
HX = get_entropy(X)
HX

0.9402859586706309

In [130]:
print('Entropia(outlook) = {}'.format(get_entropy_attrib('outlook',X)))
print('Entropia(temperature) = {}'.format(get_entropy_attrib('temperature',X)))
print('Entropia(humidity) = {}'.format(get_entropy_attrib('humidity',X)))
print('Entropia(windy) = {}'.format(get_entropy_attrib('windy',X)))
print('')
print('GI(outlook) = {}'.format(HX - get_entropy_attrib('outlook',X)))
print('GI(temperature) = {}'.format(HX - get_entropy_attrib('temperature',X)))
print('GI(humidity) = {}'.format(HX - get_entropy_attrib('humidity',X)))
print('GI(windy) = {}'.format(HX - get_entropy_attrib('windy',X)))


Entropia(outlook) = 0.6935361388961918
Entropia(temperature) = 0.9389462162661897
Entropia(humidity) = 0.8380423950607804
Entropia(windy) = 0.8921589282623617

GI(outlook) = 0.2467498197744391
GI(temperature) = 0.0013397424044412354
GI(humidity) = 0.10224356360985054
GI(windy) = 0.04812703040826927


In [95]:
arvore['outlook'] = {}
arvore

{'outlook': {}}

# Os possíveis valores para o atributo OUTLOOK são: sunny, overcast e ranny 
## Comecei pelo sunny pois é o primeiro valor que aparece no dataset

In [96]:
def obter_subconjunto(index, valor, conjunto):
    sub_conjunto = [linha for linha in conjunto if linha[index] == valor]
    return sub_conjunto

In [104]:
# Obter novo sub-conjuto
x1 = np.array(obter_subconjunto(0,'sunny',X))
x1

array([['sunny', 'maior', 'maior', 'false', 'no'],
       ['sunny', 'maior', 'maior', 'true', 'no'],
       ['sunny', 'menor', 'maior', 'false', 'no'],
       ['sunny', 'menor', 'menor', 'false', 'yes'],
       ['sunny', 'maior', 'menor', 'true', 'yes']],
      dtype='<U8')

In [132]:
# calcular a entropia do subconjunto SUNNY
HX = get_entropy(x1)
print('Entropia(temperature) = {}'.format(get_entropy_attrib('temperature',x1)))
print('Entropia(humidity) = {}'.format(get_entropy_attrib('humidity', x1)))
print('Entropia(windy) = {}'.format(get_entropy_attrib('windy', x1)))

print('')

print('GI(windy) = {}'.format(HX - get_entropy_attrib('windy',X)))
print('GI(temperature) = {}'.format(HX - get_entropy_attrib('temperature',X)))
print('GI(humidity) = {}'.format(HX - get_entropy_attrib('humidity',X)))

Entropia(temperature) = 0.9509775004326937
Entropia(humidity) = 0.0
Entropia(windy) = 0.9509775004326937

GI(windy) = 0.07879166619230693
GI(temperature) = 0.0320043781884789
GI(humidity) = 0.1329081993938882


In [147]:
arvore['outlook'] = {'sunny': {'humidity':''}}
arvore
#import json
#json.dumps(arvore, ensure_ascii=False)

{'outlook': {'sunny': {'humidity': ''}}}

# Os possíveis valores para o atributo HUMIDITY são: menor que média e maior que a média
## Comecei pelo maior pois é o primeiro valor que aparece no dataset

In [154]:
x2 = np.array(obter_subconjunto(2, 'maior', x1))
x2

array([['sunny', 'maior', 'maior', 'false', 'no'],
       ['sunny', 'maior', 'maior', 'true', 'no'],
       ['sunny', 'menor', 'maior', 'false', 'no']],
      dtype='<U8')

In [171]:
HX = get_entropy(x2)
print(HX)
print('Entropia(temperature) = {}'.format(get_entropy_attrib('temperature',x2)))
print('Entropia(windy) = {}'.format(get_entropy_attrib('windy', x2)))

print('')

print('GI(windy) = {}'.format(HX - get_entropy_attrib('windy',x2)))
print('GI(temperature) = {}'.format(HX - get_entropy_attrib('temperature',x2)))


0.0
Entropia(temperature) = 0.0
Entropia(windy) = 0.0

GI(windy) = 0.0
GI(temperature) = 0.0


### A entropia do subconjunto x2 é zero, portanto chegamos à condição de parada da recursão.
### Assim, quando a humidade for maior que a média a folha é NO, e quando for menor que a média a folha é YES.

In [172]:
arvore['outlook'] = {'sunny': {'humidity':{'maior':'NO', 'menor':'YES'}}}
arvore

{'outlook': {'sunny': {'humidity': {'maior': 'NO', 'menor': 'YES'}}}}

In [173]:
# Obter novo sub-conjuto
x1 = np.array(obter_subconjunto(0,'overcast',X))
x1

array([['overcast', 'maior', 'menor', 'false', 'yes'],
       ['overcast', 'menor', 'menor', 'true', 'yes'],
       ['overcast', 'menor', 'maior', 'true', 'yes'],
       ['overcast', 'maior', 'menor', 'false', 'yes']],
      dtype='<U8')

In [174]:
# calcular a entropia do subconjunto OVERCAST
HX = get_entropy(x1)
print(HX)
print('Entropia(temperature) = {}'.format(get_entropy_attrib('temperature',x1)))
print('Entropia(windy) = {}'.format(get_entropy_attrib('windy', x1)))

print('')

print('GI(windy) = {}'.format(HX - get_entropy_attrib('windy',x1)))
print('GI(temperature) = {}'.format(HX - get_entropy_attrib('temperature',x1)))

0.0
Entropia(temperature) = 0.0
Entropia(windy) = 0.0

GI(windy) = 0.0
GI(temperature) = 0.0


In [175]:
# A entropia de overcast é zero, logo o únivo valor dele é YES
arvore['outlook'] = {'sunny': {'humidity':{'maior':'NO', 'menor':'YES'}},
                    'overcast':'yes'}
arvore


{'outlook': {'overcast': 'yes',
  'sunny': {'humidity': {'maior': 'NO', 'menor': 'YES'}}}}

In [180]:
# Obter novo sub-conjuto
x1 = np.array(obter_subconjunto(0,'ranny',X))
x1

array([['ranny', 'menor', 'maior', 'false', 'yes'],
       ['ranny', 'menor', 'menor', 'false', 'yes'],
       ['ranny', 'menor', 'menor', 'true', 'no'],
       ['ranny', 'maior', 'menor', 'false', 'yes'],
       ['ranny', 'menor', 'menor', 'true', 'no']],
      dtype='<U8')

In [181]:
# calcular a entropia do subconjunto RANNY
HX = get_entropy(x1)
print('Entropia = {}'.format(HX))
print('Entropia(temperature) = {}'.format(get_entropy_attrib('temperature',x1)))
print('Entropia(windy) = {}'.format(get_entropy_attrib('windy', x1)))

print('')

print('GI(windy) = {}'.format(HX - get_entropy_attrib('windy',x1)))
print('GI(temperature) = {}'.format(HX - get_entropy_attrib('temperature',x1)))

Entropia = 0.9709505944546686
Entropia(temperature) = 0.8
Entropia(windy) = 0.0

GI(windy) = 0.9709505944546686
GI(temperature) = 0.17095059445466854


In [182]:
# A entropia de overcast é zero, logo o únivo valor dele é YES
arvore['outlook'] = {'sunny': {'humidity':{'maior':'NO', 'menor':'YES'}},
                    'overcast':'yes',
                    'ranny':{'windy':''}}
arvore


{'outlook': {'overcast': 'yes',
  'ranny': {'windy': ''},
  'sunny': {'humidity': {'maior': 'NO', 'menor': 'YES'}}}}

In [183]:
# Para false e tru do atributo WINDY

In [185]:
# Obter novo sub-conjuto
x2 = np.array(obter_subconjunto(3,'false',x1))
x2

array([['ranny', 'menor', 'maior', 'false', 'yes'],
       ['ranny', 'menor', 'menor', 'false', 'yes'],
       ['ranny', 'maior', 'menor', 'false', 'yes']],
      dtype='<U8')

In [186]:
# calcular a entropia do subconjunto RANNY
HX = get_entropy(x2)
print('Entropia = {}'.format(HX))
print('Entropia(temperature) = {}'.format(get_entropy_attrib('temperature',x2)))
print('Entropia(windy) = {}'.format(get_entropy_attrib('windy', x2)))

print('')

print('GI(windy) = {}'.format(HX - get_entropy_attrib('windy',x2)))
print('GI(temperature) = {}'.format(HX - get_entropy_attrib('temperature',x2)))

Entropia = 0.0
Entropia(temperature) = 0.0
Entropia(windy) = 0.0

GI(windy) = 0.0
GI(temperature) = 0.0


In [190]:
# Entropia é zero
# A entropia de overcast é zero, logo o únivo valor dele é YES
arvore['outlook'] = {
    'sunny': {
        'humidity':{
            'maior':'NO', 
            'menor':'YES'}},            
    'overcast':'yes',        
    'ranny':{
        'windy':{
            'false': 'YES',
            'true':'NO'}}}
arvore

{'outlook': {'overcast': 'yes',
  'ranny': {'windy': {'false': 'YES', 'true': 'NO'}},
  'sunny': {'humidity': {'maior': 'NO', 'menor': 'YES'}}}}